In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

import geopandas as gpd
import pandas as pd
import numpy as np
from shared_utils import utils, geography_utils

pd.options.display.max_columns = 100

import gcsfs

from calitp_data_analysis import get_fs
fs = get_fs()
from calitp_data_analysis.sql import to_snakecase

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_311/1872485643.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling P

# Read In CSIS Metrics Testing Data Entry

Process data entry tabs and save as parquets 

In [ ]:
# safety
safety_df = to_snakecase(pd.read_excel(f'{GCS_FILE_PATH}Metrics_Scoring_All_Projects.xlsx', sheet_name="Safety"))

In [ ]:
safety_df.info()

In [ ]:
safety_df.crf_2.value_counts()

In [ ]:
# replace field that's entirely space (or empty) with NaN
safety_df=safety_df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
safety_df = safety_df.astype({'crf_1':'float','crf_2':'float'})

In [ ]:
safety_df.to_parquet(f'{GCS_FILE_PATH}data_entry_raw_safety.parquet')

In [ ]:
# DAC Traffic Impacts
dac_traffic = pd.read_excel(f'{GCS_FILE_PATH}Metrics_Scoring_All_Projects.xlsx', sheet_name="DAC Traffic Impacts")

In [ ]:
dac_traffic.info()

In [ ]:
dac_traffic.to_parquet(f'{GCS_FILE_PATH}data_entry_raw_dac_traffic.parquet')

In [ ]:
# Land Use
land_use = pd.read_excel(f'{GCS_FILE_PATH}Metrics_Scoring_All_Projects.xlsx', sheet_name="Land Use")

In [ ]:
land_use.info()

In [ ]:
print(pd. __version__)

In [7]:
# VMT
vmt = to_snakecase(pd.read_excel(f'{GCS_FILE_PATH}Metrics_Scoring_All_Projects.xlsx', sheet_name="VMT"))

In [8]:
vmt.columns = vmt.columns.str.replace('?', '')

/tmp/ipykernel_311/65901047.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  vmt.columns = vmt.columns.str.replace('?', '')


In [9]:
vmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 17 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   submission_log_number                            80 non-null     int64  
 1   program                                          80 non-null     object 
 2   project_name                                     80 non-null     object 
 3   data_enterer_name                                54 non-null     object 
 4   done_y_n                                         50 non-null     object 
 5   notes                                            53 non-null     object 
 6   estimated_change_in_vmt__total_for_project_      39 non-null     object 
 7   is_specific_                                     52 non-null     object 
 8   project_contains_new_lane_miles_                 50 non-null     object 
 9   total_new_lane_miles              

In [16]:
vmt['estimated_change_in_vmt__total_for_project_'] = vmt['estimated_change_in_vmt__total_for_project_'].str.replace('\n', '')

In [22]:
vmt.total_new_lane_miles.value_counts()

0       14
N       14
?        3
3.5      1
7.8      1
16       1
17.2     1
10.4     1
7.5      1
32       1
30.4     1
4.2      1
1.3      1
22.2     1
14       1
6.7      1
4.4      1
Name: total_new_lane_miles, dtype: int64

In [23]:
vmt = vmt.astype({'project_contains_new_lane_miles_':'str', 'total_new_lane_miles':'str'})

In [24]:
vmt.to_parquet(f'{GCS_FILE_PATH}data_entry_raw_vmt.parquet')